In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import sys
sys.path.append('d:/flagellar/code/core')
sys.path.append('/kaggle/code/core')
sys.path.append('/kaggle/input/my-flg-library/')
import flg_support as fls
import flg_unet
import flg_numerics
import importlib
import numpy as np
import matplotlib.pyplot as plt
import time
import copy

In [ ]:
train_data = fls.load_all_train_data()

In [ ]:
%%time
importlib.reload(flg_unet)
#importlib.reload(fls)
model = flg_unet.UNetModel()
model.n_epochs = 1000
model.n_images_per_update = 20
model.train(train_data)
model.train_loss_list1
plt.figure()
plt.plot(model.train_loss_list1)
plt.plot(model.train_loss_list2)
fls.dill_save(fls.temp_dir + 'model.pickle', model)

In [ ]:
model_loaded = fls.dill_load(fls.temp_dir + '/model.pickle')
training_data_loaded = copy.deepcopy(train_data)
for t in training_data_loaded: t.load_to_h5py()
cpu,device = fls.prep_pytorch(0,False,False)
#model_loaded.model.eval()
#model_loaded.model.to(device);

In [ ]:
import monai
self = model_loaded
cpu,device = fls.prep_pytorch(self.seed, self.deterministic_train, True)  

criterion1 = nn.BCEWithLogitsLoss()
criterion2 = monai.losses.TverskyLoss(smooth_nr=1e-05, smooth_dr=1e-05, batch=False, to_onehot_y=False, sigmoid=True, \
                                         alpha=self.tversky_alpha, beta=self.tversky_beta)

model = copy.deepcopy(self.model)
model.eval()
model = model.to(device)

self.dataset.data_list = copy.deepcopy(train_data)
self.n_images_per_update = 10
data_loader = iter(torch.utils.data.DataLoader(self.dataset,batch_size=self.n_images_per_update,num_workers=0,pin_memory=True,persistent_workers=False))

for i_epoch in range(1):
    print(i_epoch)
    running_loss1 = 0.0
    running_loss2 = 0.0
    images, targets = next(data_loader)
    for i_image in range(images.shape[0]):
        with torch.amp.autocast('cuda'):
            with torch.no_grad():
                image_device = images[i_image:i_image+1,np.newaxis,:,:,:].to(device, dtype=torch.float32, non_blocking=True)
                target_device = targets[i_image:i_image+1,np.newaxis,:,:,:].to(device, dtype=torch.float32, non_blocking=True)
                output = model(image_device)                                    
                loss1 = criterion1(output, target_device)
                loss2 = criterion2(output, target_device)                                                
                running_loss1 += loss1
                running_loss2 += loss2
        _,ax = plt.subplots(1,3,figsize=(18,6))
        plt.sca(ax[0])
        plt.imshow(np.mean(image_device.detach().cpu().numpy()[0,0,:,:,:],axis=0), cmap='bone')
        plt.colorbar()
        plt.sca(ax[1])
        plt.imshow(np.mean(target_device.detach().cpu().numpy()[0,0,:,:,:],axis=0), cmap='bone')
        plt.colorbar()
        plt.sca(ax[2])
        plt.imshow(np.mean(torch.sigmoid(output).detach().cpu().numpy()[0,0,:,:,:],axis=0), cmap='bone')
        plt.colorbar()
        plt.suptitle(str(loss2.item()))

    epoch_loss1 = running_loss1 / images.shape[0]
    epoch_loss2 = running_loss2 / images.shape[0]
    loss = epoch_loss1 + epoch_loss2

In [ ]:
eafew